In [1]:
import pandas as pd
import os

In [2]:
file_dir = os.path.join('train_data')
file_path = os.path.join(file_dir, 'merged_data.csv')
df = pd.read_csv(file_path)

print(df.shape)
print(df.head())

(164917, 47)
  Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  \
0       CHEMBL597177           NaN                 NaN            292.29   
1      CHEMBL3687191           NaN                 NaN            453.59   
2      CHEMBL3128229           NaN                 NaN            527.43   
3      CHEMBL3128222           NaN                 NaN            502.58   
4      CHEMBL3128221           NaN                 NaN            563.48   

   #RO5 Violations  AlogP Compound Key  \
0              0.0   3.61           37   
1              0.0   2.35   BDBM144321   
2              2.0   5.70           8b   
3              1.0   4.62          10c   
4              2.0   5.24          10b   

                                              Smiles Standard Type  \
0        Nc1ccc2c(c1)/C(=C\c1ccc(-c3ccoc3)o1)C(=O)N2          IC50   
1  CCc1nc(C(N)=O)c(Nc2ccc(N3CCN(C)CC3)cc2)nc1N[C@...          IC50   
2  O=C(Nc1cccc(CNc2ncnc3c2cnn3CCc2ccccc2)c1)c1ccc...          IC50 

/tmp/ipykernel_3124084/262385642.py:3: DtypeWarning: Columns (27,28,30,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [3]:
df = df[df['Standard Units'] == 'nM'] # nM이 아닌 행 제거
df = df[df['Standard Type'] == 'IC50'] # nM이 아닌 행 제거
df = df[df['Target Organism'] == 'Homo sapiens'] # Homo sapines가 아닌 행 제거
df = df[df['Standard Relation'] == "'='"] # '='이 아닌 행 제거
df = df[df['Target Type'] == 'SINGLE PROTEIN'] # 'SINGLE PROTEIN'이 아닌 행 제거
df = df.dropna(subset=['Standard Value']) # Standard Value 비어있는 행 제거
df = df.dropna(subset=['Smiles']) # Smiles 비어있는 행 제거

# 인덱스 재설정
df.reset_index(drop=True, inplace=True)

print(df.shape)
print(df)

(125511, 47)
       Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  \
0            CHEMBL597177           NaN                 NaN            292.29   
1           CHEMBL3687191           NaN                 NaN            453.59   
2           CHEMBL3128229           NaN                 NaN            527.43   
3           CHEMBL3128222           NaN                 NaN            502.58   
4           CHEMBL3128221           NaN                 NaN            563.48   
...                   ...           ...                 ...               ...   
125506      CHEMBL5084355           NaN                 NaN            563.71   
125507      CHEMBL5078135           NaN                 NaN            563.71   
125508      CHEMBL5266361           NaN                 NaN            465.93   
125509      CHEMBL5203775           NaN                 NaN            440.53   
125510      CHEMBL5220920           NaN                 NaN            364.41   

        #RO5 V

In [4]:
# 남길 열 목록
columns_to_keep = ['Molecule ChEMBL ID', 'Target ChEMBL ID', 'Standard Value', 'Smiles', 'Target Name']

# 지정된 열만 남기고 나머지 열 제거
df = df[columns_to_keep]

print(df.shape)
print(df)


(125511, 5)
       Molecule ChEMBL ID Target ChEMBL ID  Standard Value  \
0            CHEMBL597177       CHEMBL2041         34000.0   
1           CHEMBL3687191       CHEMBL2041             1.1   
2           CHEMBL3128229       CHEMBL2041          9310.0   
3           CHEMBL3128222       CHEMBL2041          4830.0   
4           CHEMBL3128221       CHEMBL2041         12850.0   
...                   ...              ...             ...   
125506      CHEMBL5084355       CHEMBL3267             1.0   
125507      CHEMBL5078135       CHEMBL3267             1.0   
125508      CHEMBL5266361       CHEMBL3267           420.0   
125509      CHEMBL5203775       CHEMBL3267          2855.5   
125510      CHEMBL5220920       CHEMBL3267            10.0   

                                                   Smiles  \
0             Nc1ccc2c(c1)/C(=C\c1ccc(-c3ccoc3)o1)C(=O)N2   
1       CCc1nc(C(N)=O)c(Nc2ccc(N3CCN(C)CC3)cc2)nc1N[C@...   
2       O=C(Nc1cccc(CNc2ncnc3c2cnn3CCc2ccccc2)c1)c1ccc...   

In [5]:
# Smiles와 Target Name 열에서 쌍의 중복 확인
smiles_target_duplicates = df.duplicated(subset=['Molecule ChEMBL ID', 'Target ChEMBL ID'], keep=False)
duplicated_smiles_target_df = df[smiles_target_duplicates]

# 중복된 Smiles와 Target Name 쌍 그룹 지어서 출력
if not duplicated_smiles_target_df.empty:
    print("Smiles와 Target Name 열의 쌍에서 중복된 값들:")
    grouped_smiles_target = duplicated_smiles_target_df.groupby(['Molecule ChEMBL ID', 'Target ChEMBL ID'])
    for name, group in grouped_smiles_target:
        print(f"Smiles, Target Name 쌍: {name}")
        print(group)
        print("-" * 40)
else:
    print("Smiles와 Target Name 열의 쌍에 중복된 값이 없습니다.")

Smiles와 Target Name 열의 쌍에서 중복된 값들:
Smiles, Target Name 쌍: ('CHEMBL100172', 'CHEMBL267')
      Molecule ChEMBL ID Target ChEMBL ID  Standard Value  \
63815       CHEMBL100172        CHEMBL267             9.8   
63816       CHEMBL100172        CHEMBL267           140.0   

                                                  Smiles  \
63815  COCCOc1cc2c(cc1OC)Nc1ncnc(Nc3cc(OC)c(Cl)cc3Cl)...   
63816  COCCOc1cc2c(cc1OC)Nc1ncnc(Nc3cc(OC)c(Cl)cc3Cl)...   

                       Target Name  
63815  Tyrosine-protein kinase SRC  
63816  Tyrosine-protein kinase SRC  
----------------------------------------
Smiles, Target Name 쌍: ('CHEMBL100551', 'CHEMBL267')
      Molecule ChEMBL ID Target ChEMBL ID  Standard Value  \
62660       CHEMBL100551        CHEMBL267            28.0   
63759       CHEMBL100551        CHEMBL267             2.3   

                                                  Smiles  \
62660  COCCOc1cc2c(cc1OC)Cc1c(Nc3cc(OC)c(Cl)cc3Cl)ncn...   
63759  COCCOc1cc2c(cc1OC)Cc1c(Nc3cc(OC

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
# 중복된 행 제거 (첫 번째만 남기고 나머지 중복된 행 제거)
df_unique = df.drop_duplicates(subset=['Molecule ChEMBL ID', 'Target ChEMBL ID'], keep='first')

# 결과 확인
print("중복된 행 제거 후 데이터프레임:")
print(df_unique.shape)
print(df_unique.head())

중복된 행 제거 후 데이터프레임:
(90215, 5)
  Molecule ChEMBL ID Target ChEMBL ID  Standard Value  \
0       CHEMBL597177       CHEMBL2041         34000.0   
1      CHEMBL3687191       CHEMBL2041             1.1   
2      CHEMBL3128229       CHEMBL2041          9310.0   
3      CHEMBL3128222       CHEMBL2041          4830.0   
4      CHEMBL3128221       CHEMBL2041         12850.0   

                                              Smiles  \
0        Nc1ccc2c(c1)/C(=C\c1ccc(-c3ccoc3)o1)C(=O)N2   
1  CCc1nc(C(N)=O)c(Nc2ccc(N3CCN(C)CC3)cc2)nc1N[C@...   
2  O=C(Nc1cccc(CNc2ncnc3c2cnn3CCc2ccccc2)c1)c1ccc...   
3  O=S(=O)(Nc1cccc(CNc2ncnc3c2cnn3CCc2ccccc2)c1)c...   
4  O=S(=O)(Nc1cccc(CNc2ncnc3c2cnn3CCc2ccccc2)c1)c...   

                            Target Name  
0  Tyrosine-protein kinase receptor RET  
1  Tyrosine-protein kinase receptor RET  
2  Tyrosine-protein kinase receptor RET  
3  Tyrosine-protein kinase receptor RET  
4  Tyrosine-protein kinase receptor RET  


In [7]:
# 남길 열 목록
columns_to_keep = ['Molecule ChEMBL ID', 'Smiles', 'Target Name']

# 지정된 열만 남기고 나머지 열 제거
df = df[columns_to_keep]

In [8]:
new_column_order = ['Smiles', 'Target Name', 'Standard Value']
df = df_unique[new_column_order]

# 빈 열 추가
df['Smiles_feature_vector'] = pd.Series([None] * len(df))
df['image_feature_vector'] = pd.Series([None] * len(df))
df['target_protein_vector'] = pd.Series([None] * len(df))

# 결과 확인
print(df)

                                                   Smiles  \
0             Nc1ccc2c(c1)/C(=C\c1ccc(-c3ccoc3)o1)C(=O)N2   
1       CCc1nc(C(N)=O)c(Nc2ccc(N3CCN(C)CC3)cc2)nc1N[C@...   
2       O=C(Nc1cccc(CNc2ncnc3c2cnn3CCc2ccccc2)c1)c1ccc...   
3       O=S(=O)(Nc1cccc(CNc2ncnc3c2cnn3CCc2ccccc2)c1)c...   
4       O=S(=O)(Nc1cccc(CNc2ncnc3c2cnn3CCc2ccccc2)c1)c...   
...                                                   ...   
125506  Cc1nc(-c2cccc(Nc3nc(C)c(-c4cc5c(c(S(C)(=O)=O)c...   
125507  Cc1nc(Nc2cccc(-c3nc(C)n(C)n3)n2)sc1-c1cc2c(c(S...   
125508  Cn1nccc1-n1c([C@@H]2CSCN2c2ncnc3[nH]cnc23)nc2c...   
125509  COc1cc(-c2cc3c(O[C@H]4CCN(C(=O)C5CCC5)C4)ncnc3...   
125510    COc1cccc2cc(-c3nn(C4CCOCC4)c4ncnc(N)c34)[nH]c12   

                                 Target Name  Standard Value  \
0       Tyrosine-protein kinase receptor RET         34000.0   
1       Tyrosine-protein kinase receptor RET             1.1   
2       Tyrosine-protein kinase receptor RET          9310.0   
3       Tyr

/tmp/ipykernel_3124084/3598440930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Smiles_feature_vector'] = pd.Series([None] * len(df))


In [9]:
df.to_csv('train_data/train_data.csv', index = False)

train sets -> valdiation sets

In [13]:
file_dir = os.path.join('train_data','original_data')
file_path = os.path.join(file_dir, 'train.csv')
df = pd.read_csv(file_path)

print(df.shape)
print(df.head())

(1952, 15)
  Molecule ChEMBL ID Standard Type Standard Relation  Standard Value  \
0      CHEMBL4443947          IC50               '='           0.022   
1      CHEMBL4556091          IC50               '='           0.026   
2      CHEMBL4566431          IC50               '='           0.078   
3      CHEMBL4545898          IC50               '='           0.081   
4      CHEMBL4448950          IC50               '='           0.099   

  Standard Units  pChEMBL Value Assay ChEMBL ID Target ChEMBL ID  \
0             nM          10.66   CHEMBL4361896       CHEMBL3778   
1             nM          10.59   CHEMBL4345131       CHEMBL3778   
2             nM          10.11   CHEMBL4345131       CHEMBL3778   
3             nM          10.09   CHEMBL4345131       CHEMBL3778   
4             nM          10.00   CHEMBL4361896       CHEMBL3778   

                                  Target Name Target Organism     Target Type  \
0  Interleukin-1 receptor-associated kinase 4    Homo sapiens  SIN

In [14]:
# 남길 열 목록
columns_to_keep = ['Smiles', 'Target Name', 'Standard Value']

# 지정된 열만 남기고 나머지 열 제거
df = df[columns_to_keep]

new_column_order = ['Smiles', 'Target Name', 'Standard Value']
df = df[new_column_order]

# 빈 열 추가
df['Smiles_feature_vector'] = pd.Series([None] * len(df))
df['image_feature_vector'] = pd.Series([None] * len(df))
df['target_protein_vector'] = pd.Series([None] * len(df))

# 결과 확인
print(df)

                                                 Smiles  \
0     CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...   
1     CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
2     CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
3     CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
4     COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...   
...                                                 ...   
1947      O=C(Nc1nc2cc[nH]cc-2n1)c1cccc([N+](=O)[O-])c1   
1948              CCCCn1c(NC(=O)c2cccc(Cl)c2)nc2ccccc21   
1949  O=C(Nc1nc2cc(F)c(F)cc2[nH]1)c1cccc([N+](=O)[O-...   
1950  OC[C@H]1C[C@@H](Nc2nc(Nc3ccccc3)ncc2-c2nc3cccc...   
1951                               CC(C)Oc1ccccc1C(N)=O   

                                     Target Name  Standard Value  \
0     Interleukin-1 receptor-associated kinase 4           0.022   
1     Interleukin-1 receptor-associated kinase 4           0.026   
2     Interleukin-1 receptor-associated kinase 4           0.078   
3     Interleukin-1

In [15]:
df.to_csv('train_data/val_data.csv', index = False)